In [90]:
using ZipFile, Distributed, CSV, DataFrames, Random, StatsBase, Plots, Statistics, StableRNGs, MLJBase
using Flux
using Flux: logitbinarycrossentropy
using BSON: @save
using Distributions
using PyCall
PyCall.libpython

file = CSV.read("train.csv", DataFrame)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep
,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Float64
1,842,0,2.2,0,1,0,7,0.6
2,1021,1,0.5,1,0,1,53,0.7
3,563,1,0.5,1,2,1,41,0.9
4,615,1,2.5,0,0,0,10,0.8
5,1821,1,1.2,0,13,1,44,0.6
6,1859,0,0.5,1,3,0,22,0.7
7,1821,0,1.7,0,4,1,10,0.8
8,1954,0,0.5,1,0,0,24,0.8
9,1445,1,0.5,0,0,0,53,0.7


In [91]:
# Transfer DataFrame to matrix form
df = file |> Tables.matrix

# Transfer the dataset to 2-classifiers. df_0 represents the result is 0, df_1 represents the result is 1.
# Due to my datasset is binary problems, and each result is 50 percents of the whole dataset. So i didn't add any other pre-actions for dataset. 

mat_0 = df[df[:,21] .<= 1, :]
mat_1 = df[df[:,21] .>= 2, :]


1000×21 Matrix{Float64}:
 1021.0  1.0  0.5  1.0   0.0  1.0  53.0  …   3.0   7.0  1.0  1.0  0.0  2.0
  563.0  1.0  0.5  1.0   2.0  1.0  41.0      2.0   9.0  1.0  1.0  0.0  2.0
  615.0  1.0  2.5  0.0   0.0  0.0  10.0      8.0  11.0  1.0  0.0  0.0  2.0
 1821.0  0.0  1.7  0.0   4.0  1.0  10.0      8.0  18.0  1.0  0.0  1.0  3.0
  769.0  1.0  2.9  1.0   0.0  0.0   9.0      2.0   7.0  0.0  0.0  0.0  3.0
 1520.0  1.0  2.2  0.0   5.0  1.0  33.0  …   9.0  13.0  1.0  1.0  1.0  3.0
  803.0  1.0  2.1  0.0   7.0  0.0  17.0      1.0   4.0  1.0  0.0  1.0  2.0
  838.0  0.0  0.5  0.0   1.0  1.0  13.0      9.0  19.0  1.0  0.0  1.0  3.0
  595.0  0.0  0.9  1.0   7.0  1.0  23.0      2.0  18.0  1.0  1.0  0.0  3.0
  772.0  0.0  1.1  1.0  12.0  0.0  39.0     15.0   3.0  1.0  1.0  0.0  3.0
    ⋮                         ⋮          ⋱   ⋮                         ⋮
 1784.0  0.0  1.6  0.0   4.0  0.0  41.0      1.0  11.0  0.0  1.0  0.0  2.0
 1262.0  0.0  1.8  1.0  12.0  0.0  34.0      3.0   4.0  0.0  1.0  1.0  2.0
 1

In [92]:
# randomly select 70% data from data set as the train data

train_data = randsubseq(1:1000, 0.7)
train_df = vcat(mat_0[train_data, :], mat_1[train_data, :])

# select the rest as the test data 

test_data = [i for i in 1:1000 if isempty(searchsorted(train_data, i))]
test_df = vcat(mat_0[test_data, :], mat_1[test_data, :])

596×21 Matrix{Float64}:
  842.0  0.0  2.2  0.0   1.0  0.0   7.0  …   7.0  19.0  0.0  0.0  1.0  1.0
  509.0  1.0  0.6  1.0   2.0  1.0   9.0     10.0  12.0  1.0  0.0  0.0  0.0
 1815.0  0.0  2.8  0.0   2.0  0.0  33.0      0.0   2.0  1.0  0.0  0.0  1.0
  775.0  0.0  1.0  0.0   3.0  0.0  46.0     15.0  11.0  1.0  1.0  1.0  0.0
  682.0  1.0  0.5  0.0   4.0  0.0  19.0      1.0  18.0  0.0  1.0  1.0  1.0
 1602.0  1.0  2.8  1.0   4.0  1.0  38.0  …   7.0  20.0  1.0  0.0  0.0  0.0
  851.0  0.0  0.5  0.0   3.0  0.0  21.0      7.0  10.0  1.0  0.0  1.0  0.0
  644.0  1.0  2.7  0.0   0.0  0.0  22.0      1.0  15.0  1.0  0.0  0.0  0.0
  589.0  1.0  2.3  1.0   1.0  0.0  61.0      7.0   2.0  1.0  0.0  1.0  1.0
 1656.0  0.0  1.0  0.0   5.0  1.0  34.0     12.0  14.0  1.0  1.0  0.0  1.0
    ⋮                         ⋮          ⋱   ⋮                         ⋮
 1872.0  1.0  1.9  0.0   5.0  0.0  35.0      5.0  19.0  0.0  0.0  0.0  3.0
  772.0  0.0  1.0  1.0   0.0  1.0  23.0      0.0  17.0  1.0  0.0  0.0  2.0
 17

In [93]:
# parameters of the training and testing data
para_train = train_df[:, 1:20]
para_test = test_df[:, 1:20]

# value is never used?
value_train = train_df[:, 21]
value_test = test_df[:, 21]
value_train_std = reshape(value_train, (1, :)) 
value_test_std = reshape(value_test, (1, :)) 


1×596 Matrix{Float64}:
 1.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  …  2.0  2.0  3.0  3.0  2.0  2.0  3.0

In [94]:
dt = StatsBase.fit(ZScoreTransform, para_train, dims=1)

# rotate the matrix for 90 degree
para_train_std = StatsBase.transform(dt, para_train)
para_test_std = StatsBase.transform(dt, para_test)

para_train_std = transpose(para_train_std) 
para_test_std = transpose(para_test_std)

20×596 transpose(::Matrix{Float64}) with eltype Float64:
 -0.912305  -1.67573     1.31836    …   1.54303     0.050572   1.35046
 -0.961908   1.03886    -0.961908       1.03886    -0.961908   1.03886
  0.833985  -1.14621     1.57656        0.338936    0.338936   0.710222
 -1.00822    0.991136   -1.00822       -1.00822     0.991136  -1.00822
 -0.750252  -0.52508    -0.52508       -0.975423    1.72664    0.82595
 -1.02708    0.972944   -1.02708    …  -1.02708    -1.02708   -1.02708
 -1.36189   -1.2502      0.0900075     -0.133361    0.14585    1.5419
  0.323994  -1.39127     0.323994       0.323994   -1.39127   -1.39127
  1.40323   -1.33339     0.567842      -0.814872    0.279776  -1.391
 -1.11044    0.196398   -0.239215       0.196398    0.196398   0.196398
 -1.29691    0.84625     1.17597    …  -0.967193    1.01111    0.84625
 -1.40921    1.11531    -0.0825366      0.0711493  -0.95041    1.83854
 -1.15903   -0.0765339  -1.17753       -1.19141    -1.20297    1.53101
  0.386513  -1.49124 

In [95]:
# import xcs package
a = pyimport("xcs")

PyObject <module 'xcs' from '/Users/yuchen/.julia/conda/3/lib/python3.9/site-packages/xcs/__init__.py'>

In [96]:
#set variables
PreClassifiedData = a.scenarios.PreClassifiedData
XCSAlgorithm = a.XCSAlgorithm

PyObject <class 'xcs.XCSAlgorithm'>

In [97]:
# we'll create the algorithm which will be used to manage the classifier set and learn the mapping defined by the problem we have selected
al = XCSAlgorithm()

PyObject <xcs.XCSAlgorithm object at 0x1acab5430>

In [98]:
al.exploration_probability = .1
al.discount_factor = 0
al.do_ga_subsumption = true
al.do_action_set_subsumption = true

true

In [99]:
#output the model using py style
py"""
def outputmodel(x):
    import logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.info('Classifiers:\n\n%s\n', x)

"""

In [100]:
scenarioTrain = PreClassifiedData(para_train,value_train)
scenarioTest = PreClassifiedData(para_test,value_test)

PyObject <xcs.scenarios.PreClassifiedData object at 0x1875d64f0>

In [101]:
model_train = al.new_model(scenarioTrain)
model_test = al.new_model(scenarioTest)

PyObject <xcs.ClassifierSet object at 0x1ac99a8b0>

In [102]:
#run the model
# model_train = al.run(scenarioTrain)
# model_test = al.run(scenarioTest)
model_train.run(scenarioTrain, learn = true)
model_test.run(scenarioTest, learn = true)

In [103]:
#train model output
py"outputmodel"(model_train)

INFO:__main__:Classifiers:

1##10111##1#111011#1 => 3.0
    Time Stamp: 371
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
111100#11#0111##11#1 => 0.0
    Time Stamp: 427
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1011##1#11011##11101 => 3.0
    Time Stamp: 480
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
10##00111#011##111#0 => 2.0
    Time Stamp: 629
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#0#0#1####01#1101##0 => 1.0
    Time Stamp: 651
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
10#101###101##11#10# => 3.0
    Time Stamp: 660
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0

In [104]:
#test model output
py"outputmodel"(model_test)

INFO:__main__:Classifiers:

10#100###1#1111#11## => 3.0
    Time Stamp: 37
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1111#011##1##1##1110 => 1.0
    Time Stamp: 58
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1111011#11111110#100 => 0.0
    Time Stamp: 87
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
10##00111###11#11#01 => 1.0
    Time Stamp: 99
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
11#0101111#1111#1000 => 3.0
    Time Stamp: 164
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
10#010111##1111#1001 => 3.0
    Time Stamp: 177
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
   